# PRMT-1921 CCG Failure Rates

Since the practices within a specific CCG have been digitising their Lloyd George (LG) records, they have anecdotally reported more failures.

We want to look into the data to:
- Understand if failures on transfers out have increased
- Understand why these are failing
- Understand the supplier pathways for these failures

### Scope

- Look at data for Sept 2020 - Feb 2021
- Look at all transfers out for the practices listed below
- Show overall failure rate per month so that we can see if the % failures has been increasing since 2020 
- Show a breakdown of failures per supplier pathway

In [1]:
import pandas as pd

In [2]:
error_code_lookup_file = pd.read_csv("https://raw.githubusercontent.com/nhsconnect/prm-gp2gp-data-sandbox/master/data/gp2gp_response_codes.csv")

In [3]:
transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-3/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))


asid_lookup_file = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Mar-2021.csv.gz"
asid_lookup = pd.read_csv(asid_lookup_file)

In [4]:
# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

lookup = asid_lookup[["ASID", "MName", "NACS"]]

transfers = transfers_raw.copy()
transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left').drop("NACS", axis=1)
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [5]:
list_lg_practices = ["P81006","P81037","P81129","P81742","P81086","P81157","P81668","P81059","P81077","P81128","P81149","P81150","P81031","P81191","P81089","P81737","P81087","P81079","P81133"]

transfers["is_lg_pratices"] = transfers["sending_ods_code"].apply(lambda x: x in list_lg_practices)
lg_transfers = transfers.loc[transfers["is_lg_pratices"]].copy()
lg_transfers["sending_supplier"].value_counts()

EMIS    3791
Name: sending_supplier, dtype: int64

## Monthly GP2GP failure rate for LG practices

In [6]:
lg_transfers["month_requested"] = lg_transfers["date_requested"].dt.month
lg_transfers["year_requested"] = lg_transfers["date_requested"].dt.year
lg_monthly_outcomes = pd.pivot_table(lg_transfers, index=["month_requested", "year_requested"], columns="status", values="conversation_id", aggfunc="count")
lg_monthly_outcomes = lg_monthly_outcomes.fillna(0).astype(int)
lg_monthly_outcomes = lg_monthly_outcomes.sort_values(by=["year_requested", "month_requested"])
lg_monthly_outcomes

,status,FAILED,INTEGRATED,PENDING,PENDING_WITH_ERROR
month_requested,year_requested,,,,
9,2020,7,735,9,5
10,2020,5,681,5,6
11,2020,7,525,6,6
12,2020,4,473,11,0
1,2021,3,617,10,4
2,2021,6,656,7,3


### Error Codes

In [7]:
error_columns = ["intermediate_error_codes", "final_error_code", "sender_error_code"]

lg_date_error_code_data = lg_transfers[lg_transfers["status"] != "INTEGRATED"].copy()
lg_date_error_code_data = lg_date_error_code_data.loc[:, ["month_requested", "year_requested", "conversation_id", "intermediate_error_codes", "final_error_code", "sender_error_code"]]
lg_date_error_code_data = lg_date_error_code_data.explode("intermediate_error_codes")
lg_date_error_code_data["intermediate_error_codes"].value_counts()
lg_date_error_code_data = lg_date_error_code_data.fillna("None")
lg_date_error_code_data[error_columns] = lg_date_error_code_data[error_columns].replace(error_code_lookup_file["ErrorCode"].values, error_code_lookup_file["ErrorName"].values)

In [8]:
lg_error_code_counts = pd.pivot_table(lg_date_error_code_data, index=["month_requested", "year_requested"], columns=error_columns, values="conversation_id", aggfunc="count")
lg_error_code_counts = lg_error_code_counts.fillna(0).astype(int)
lg_error_code_counts = lg_error_code_counts.sort_values(by=["year_requested", "month_requested"])
lg_error_code_counts.T

month_requested                                                     9    10  \
year_requested                                                    2020 2020   
intermediate_error_codes final_error_code    sender_error_code                
LM reassembly            Missing LM          Spine error             0    1   
None                     ABA wrong patient   None                    0    0   
                         Duplicate EHR       None                    2    1   
                         Failed to integrate None                    2    0   
                         LM general failure  None                    1    2   
                         Missing LM          None                    0    0   
                         None                Failed to generate      1    0   
                                             None                    9    5   
                                             Req not LM compliant    3    5   
                                             Spine error             1    1   
                                             Unauthorised req        0    0   
                                             Unexpected              0    0   
                         Timeout             None                    0    1   
                         Unexpected          None                    2    0   
                         Wrong patient       None                    0    0   

month_requested                                                     11   12  \
year_requested                                                    2020 2020   
intermediate_error_codes final_error_code    sender_error_code                
LM reassembly            Missing LM          Spine error             0    0   
None                     ABA wrong patient   None                    0    0   
                         Duplicate EHR       None                    6    2   
                         Failed to integrate None                    0    1   
                         LM general failure  None                    0    1   
                         Missing LM          None                    0    0   
                         None                Failed to generate      1    0   
                                             None                    6   11   
                                             Req not LM compliant    3    0   
                                             Spine error             1    0   
                                             Unauthorised req        0    0   
                                             Unexpected              1    0   
                         Timeout             None                    0    0   
                         Unexpected          None                    1    0   
                         Wrong patient       None                    0    0   

month_requested                                                     1    2   
year_requested                                                    2021 2021  
intermediate_error_codes final_error_code    sender_error_code               
LM reassembly            Missing LM          Spine error             0    3  
None                     ABA wrong patient   None                    0    1  
                         Duplicate EHR       None                    1    1  
                         Failed to integrate None                    0    0  
                         LM general failure  None                    0    1  
                         Missing LM          None                    0    1  
                         None                Failed to generate      0    1  
                                             None                   10    7  
                                             Req not LM compliant    1    0  
                                             Spine error             2    2  
                                             Unauthorised req        1    0  
                                             Unexpected       

## By Supplier Breakdown

In [9]:
lg_supplier_outcomes = pd.pivot_table(lg_transfers, index="requesting_supplier", columns="status", values="conversation_id", aggfunc="count")
lg_supplier_outcomes = lg_supplier_outcomes.fillna(0).astype(int)
lg_supplier_outcomes

status,FAILED,INTEGRATED,PENDING,PENDING_WITH_ERROR
requesting_supplier,,,,
EMIS,12,3276,36,10
TPP,19,391,12,1
Unknown,0,2,0,0
Vision,1,18,0,13


### Error codes

In [10]:
lg_supplier_error_code_data = lg_transfers[lg_transfers["status"] != "INTEGRATED"].copy()
lg_supplier_error_code_data = lg_supplier_error_code_data.loc[:, ["requesting_supplier", "conversation_id", "intermediate_error_codes", "final_error_code", "sender_error_code"]]
lg_supplier_error_code_data = lg_supplier_error_code_data.explode("intermediate_error_codes")
lg_supplier_error_code_data["intermediate_error_codes"].value_counts()
lg_supplier_error_code_data = lg_supplier_error_code_data.fillna("None")
lg_supplier_error_code_data = lg_supplier_error_code_data.replace(error_code_lookup_file["ErrorCode"].values, error_code_lookup_file["ErrorName"].values)

In [11]:
lg_supplier_error_code_counts = pd.pivot_table(lg_supplier_error_code_data, index="requesting_supplier", columns=error_columns, values="conversation_id", aggfunc="count")
lg_supplier_error_code_counts = lg_supplier_error_code_counts.fillna(0).astype(int)
lg_supplier_error_code_counts.T

requesting_supplier                                                EMIS  TPP  \
intermediate_error_codes final_error_code    sender_error_code                 
LM reassembly            Missing LM          Spine error              4    0   
None                     ABA wrong patient   None                     1    0   
                         Duplicate EHR       None                     3    9   
                         Failed to integrate None                     0    3   
                         LM general failure  None                     0    5   
                         Missing LM          None                     0    1   
                         None                Failed to generate       2    0   
                                             None                    36   12   
                                             Req not LM compliant     0    0   
                                             Spine error              6    1   
                                             Unauthorised req         1    0   
                                             Unexpected               1    0   
                         Timeout             None                     1    0   
                         Unexpected          None                     4    0   
                         Wrong patient       None                     0    1   

requesting_supplier                                                Vision  
intermediate_error_codes final_error_code    sender_error_code             
LM reassembly            Missing LM          Spine error                0  
None                     ABA wrong patient   None                       0  
                         Duplicate EHR       None                       1  
                         Failed to integrate None                       0  
                         LM general failure  None                       0  
                         Missing LM          None                       0  
                         None                Failed to generate         1  
                                             None                       0  
                                             Req not LM compliant      12  
                                             Spine error                0  
                                             Unauthorised req           0  
                                             Unexpected                 0  
                         Timeout             None                       0  
                         Unexpected          None                       0  
                         Wrong patient       None                       0